# Importing Important Library

In [ ]:
#LOAD DEPENDENCIES
import os
import time
import logging
import itertools
# import scikitplot
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects

# from keras_flops import get_flops
# from keras.utils.layer_utils import count_params
from sklearn.metrics import accuracy_score, mean_squared_error, mean_squared_log_error, classification_report, confusion_matrix, roc_curve, auc

from tensorflow.keras.models import Model
from tensorflow.keras import applications
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Add, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Layer, ReLU, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Add, Concatenate, Dropout

#PREVENT ERROR UNCESSARY MESSAGES
tf.get_logger().setLevel(logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

print("All packages loaded successfully!")

All packages loaded successfully!


#Reading Data

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_path = "F:/Willey/TGCA Extended Dataset-2/Training"
test_path = "F:/Willey/TGCA Extended Dataset-2/Test"
val_path = "F:/Willey/TGCA Extended Dataset-2/Validation"
batch_size = 16
img_height = 224
img_width = 224
no_of_classes = 4
classes_name = [0,1,2,3]
input_shape = (img_height , img_width , 3)


random_seed = np.random.seed(1142)

datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    horizontal_flip = False,
    vertical_flip = False,
    #validation_split = 0.1,
    featurewise_std_normalization=True)

train_generator = datagen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle = True,
    class_mode='categorical')

validation_generator = datagen.flow_from_directory(
    val_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle = True,
    class_mode='categorical')


print(train_generator.class_indices)

Found 3769 images belonging to 4 classes.
Found 632 images belonging to 4 classes.
{'KICH': 0, 'KIRC': 1, 'KIRP': 2, 'Normal': 3}


#Proposed Fused_MirrorNet Architecture

In [ ]:
import tensorflow as tf
import time

from tensorflow.keras.layers import Input, Add, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau


# ===================== MODEL INPUT (FIXED) =====================
model_input = Input(shape=(224, 224, 3), name='model_input')


# ===================== BRANCH A =====================
def InceptionResNetV2_tiny_A_builder(model_input):
    base_model = tf.keras.applications.InceptionResNetV2(
        weights='imagenet',
        include_top=False,
        input_tensor=model_input
    )

    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.layers[-183].output
    model = Model(
        inputs=base_model.input,
        outputs=x,
        name='InceptionResNetV2_tiny_A'
    )
    return model


InceptionResNetV2_tiny_A = InceptionResNetV2_tiny_A_builder(model_input)
print("InceptionResNetV2-Tiny-A successfully built!")


# ===================== BRANCH B =====================
def InceptionResNetV2_tiny_B_builder(model_input):
    base_model = tf.keras.applications.InceptionResNetV2(
        weights='imagenet',
        include_top=False,
        input_tensor=model_input
    )

    for layer in base_model.layers:
        layer.trainable = True
        layer._name = layer._name + "_mirror"

    x = base_model.layers[-183].output
    model = Model(
        inputs=base_model.input,
        outputs=x,
        name='InceptionResNetV2_tiny_B'
    )
    return model


InceptionResNetV2_tiny_B = InceptionResNetV2_tiny_B_builder(model_input)
print("InceptionResNetV2-Tiny-B successfully built!")


# ===================== BUILD MODELS =====================
models = [InceptionResNetV2_tiny_A, InceptionResNetV2_tiny_B]
print("Models ready for fusion")


# ===================== FUSION =====================
def fused_InceptionResNetV2_tiny(models, model_input):
    outputs = [m.output for m in models]
    y = Add()(outputs)
    y = GlobalAveragePooling2D()(y)
    y = Dense(512, activation='relu')(y)
    y = Dropout(0.5)(y)
    prediction = Dense(4, activation='softmax', name='Softmax_Classifier')(y)
    return Model(model_input, prediction, name='fused_InceptionResNetV2_tiny')


fused_InceptionResNetV2 = fused_InceptionResNetV2_tiny(models, model_input)
print("Fused-InceptionResNetV2-Tiny built successfully!")


# ===================== FINAL MODEL =====================
batch_size = 16
epochs = 40

optimizer = Adam(learning_rate=0.0001)

fused_InceptionResNetV2.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

fused_InceptionResNetV2.summary()


InceptionResNetV2-Tiny-A successfully built!
InceptionResNetV2-Tiny-B successfully built!
Models ready for fusion
Fused-InceptionResNetV2-Tiny built successfully!
Model: "fused_InceptionResNetV2_tiny"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
model_input_mirror (InputLayer) [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         model_input_mirror[0][0]         
__________________________________________________________________________________________________
conv2d_203_mirror (Conv2D)      (None, 111, 111, 32) 864         model_input_mirror[0][0]         
__________________________________________________________________________________________________
batch_n

#Training

In [ ]:
reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',   # ✅ FIXED
    factor=0.5,
    patience=2,
    verbose=1,
    mode='max',
    min_lr=1e-6
)

check_path = 'F:/Willey/Weight-7.keras'
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    check_path,
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='max'
)

callbacks = [reduce_lr, checkpoint]
nb_train_samples = train_generator.samples
nb_validation_samples = validation_generator.samples

start_time = time.time()

history = fused_InceptionResNetV2.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=callbacks,
    verbose=1
)

elapsed_time = time.time() - start_time
print("Elapsed time:", time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
print("Training completed successfully!")


C:\Users\chanchal\miniconda3\envs\chanchal\lib\site-packages\keras_preprocessing\image\image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\chanchal\miniconda3\envs\chanchal\lib\site-packages\keras_preprocessing\image\image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/40
235/235 [==============================] - 226s 752ms/step - loss: 0.6855 - accuracy: 0.7397 - val_loss: 0.4237 - val_accuracy: 0.8237

Epoch 00001: val_accuracy improved from -inf to 0.82372, saving model to F:/Willey\Weight-7.keras
Epoch 2/40
235/235 [==============================] - 143s 607ms/step - loss: 0.3389 - accuracy: 0.8854 - val_loss: 0.3121 - val_accuracy: 0.8910

Epoch 00002: val_accuracy improved from 0.82372 to 0.89103, saving model to F:/Willey\Weight-7.keras
Epoch 3/40
235/235 [==============================] - 110s 468ms/step - loss: 0.2126 - accuracy: 0.9297 - val_loss: 0.4219 - val_accuracy: 0.8574

Epoch 00003: val_accuracy did not improve from 0.89103
Epoch 4/40
235/235 [==============================] - 90s 381ms/step - loss: 0.1385 - accuracy: 0.9566 - val_loss: 0.4777 - val_accuracy: 0.8542

Epoch 00004: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.

Epoch 00004: val_accuracy did not improve from 0.89103
Epoch 5/40
235/235 [===

In [ ]:
np.save('F:/Willey/tcga_5.npy',history.history)

In [ ]:
fused_InceptionResNetV2.load_weights('F:/Willey/Training/best/Weight-7.keras')

#Learning Curves

In [ ]:
import numpy as np
history = np.load('F:/Willey/Training/best/tcga_5.npy',allow_pickle='TRUE').item()

In [1]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'] , label = 'train_loss')
plt.plot(history.history['val_loss'] , label = 'val_loss')
plt.legend()
plt.xlabel("No. of epochs")
plt.ylabel("Loss(Categorical Crossentropy)")
plt.title("Loss vs Epoch")
# plt.savefig('E:/save-2/R1_loss.png')
plt.show()
#plt.savefig('90.98_loss.png')
plt.plot(history.history['acc'] , label = 'train_acc')
plt.plot(history.history['val_acc'] , label = 'val_acc')
plt.legend()
plt.xlabel("No. of epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy vs Epoch ")
# plt.savefig('E:/save-2/R1_accuracy.png')
plt.show()

#Prediction

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_d = ImageDataGenerator(rescale=1. / 255)
test = test_d.flow_from_directory(
    'F:/Willey/TGCA Extended Dataset-2/Test',
    target_size=(224,224),
    batch_size=1,
    shuffle = False,
    class_mode='categorical')

Found 632 images belonging to 4 classes.


In [ ]:
import numpy as np
test_step = test.n//test.batch_size
test.reset()
pred = fused_InceptionResNetV2.predict(test, steps=test_step, verbose=1)
pred_class_indices = np.argmax(pred,axis=1)

## printing predicted labels
print(pred_class_indices)

632/632 [==============================] - 71s 92ms/step
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 3 0 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 2 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 3 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 0 2 2 0 2 0 3 3 2 2 3 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 3 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 0 2 2 2 0 2 2 2 2 2 2 2 2 2 1 2 0 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 

#Performance Measures

In [ ]:
from sklearn.metrics import *
classes = [0,1,2,3]


for cl in classes:

    print("class: ",cl)

    a1 = np.uint8(test.labels == cl)
    a2 = np.uint8(pred_class_indices == cl)

    print('Accuracy {}'.format(accuracy_score(y_true=a1, y_pred=a2)))
    print('F1 {}'.format(f1_score(y_true=a1, y_pred=a2)))
    print('precision {}'.format(precision_score(y_true=a1, y_pred=a2)))
    print('recall {}'.format(recall_score(y_true=a1, y_pred=a2)))

    print('jaccard {}'.format(jaccard_score(y_true=a1, y_pred=a2)))
    print("_______________________________")

class:  0
Accuracy 0.9699367088607594
F1 0.9400630914826499
precision 0.9371069182389937
recall 0.9430379746835443
jaccard 0.8869047619047619
_______________________________
class:  1
Accuracy 0.9794303797468354
F1 0.9584664536741213
precision 0.967741935483871
recall 0.9493670886075949
jaccard 0.9202453987730062
_______________________________
class:  2
Accuracy 0.939873417721519
F1 0.8805031446540881
precision 0.875
recall 0.8860759493670886
jaccard 0.7865168539325843
_______________________________
class:  3
Accuracy 0.9651898734177216
F1 0.930379746835443
precision 0.930379746835443
recall 0.930379746835443
jaccard 0.8698224852071006
_______________________________


In [ ]:
print('Accuracy {}'.format(accuracy_score(y_true=test.labels, y_pred=pred_class_indices)))
print('F1 {}'.format(f1_score(y_true=test.labels, y_pred=pred_class_indices,average = "macro")))
print('precision {}'.format(precision_score(y_true=test.labels, y_pred=pred_class_indices,average = "macro")))
print('recall {}'.format(recall_score(y_true=test.labels, y_pred=pred_class_indices,average = "macro")))

print('jaccard {}'.format(jaccard_score(y_true=test.labels, y_pred=pred_class_indices,average = "macro")))
print('confusion_matrix\n {}'.format(confusion_matrix(y_true=test.labels, y_pred=pred_class_indices)))
print('classification_report\n {}'.format(classification_report(y_true=test.labels, y_pred=pred_class_indices)))
print('\n\n')

Accuracy 0.9272151898734177
F1 0.9273531091615755
precision 0.927557150139577
recall 0.9272151898734177
jaccard 0.8658723749543633
confusion_matrix
 [[149   0   6   3]
 [  0 150   6   2]
 [  9   3 140   6]
 [  1   2   8 147]]
classification_report
               precision    recall  f1-score   support

           0       0.94      0.94      0.94       158
           1       0.97      0.95      0.96       158
           2       0.88      0.89      0.88       158
           3       0.93      0.93      0.93       158

    accuracy                           0.93       632
   macro avg       0.93      0.93      0.93       632
weighted avg       0.93      0.93      0.93       632






#Confidence Interval

In [ ]:
import numpy as np

def bootstrap_ci(metric_fn, y_true, y_pred, n_bootstrap=1000, seed=42):
    rng = np.random.default_rng(seed)
    scores = []

    for _ in range(n_bootstrap):
        idx = rng.choice(len(y_true), size=len(y_true), replace=True)
        if len(np.unique(y_true[idx])) < 2:
            continue
        scores.append(metric_fn(y_true[idx], y_pred[idx]))

    scores = np.array(scores)
    return (
        scores.mean(),
        np.percentile(scores, 2.5),
        np.percentile(scores, 97.5)
    )


In [ ]:
classes = [0, 1, 2, 3]

for cl in classes:
    print(f"\nClass {cl}")

    y_true_bin = np.uint8(test.labels == cl)
    y_pred_bin = np.uint8(pred_class_indices == cl)

    acc = bootstrap_ci(accuracy_score, y_true_bin, y_pred_bin)
    prec = bootstrap_ci(lambda y, p: precision_score(y, p, zero_division=0),
                         y_true_bin, y_pred_bin)
    rec = bootstrap_ci(lambda y, p: recall_score(y, p, zero_division=0),
                        y_true_bin, y_pred_bin)
    f1 = bootstrap_ci(lambda y, p: f1_score(y, p, zero_division=0),
                       y_true_bin, y_pred_bin)
    kappa = bootstrap_ci(cohen_kappa_score,
                          y_true_bin, y_pred_bin)

    print(f"Accuracy : {acc[0]:.3f} [{acc[1]:.3f}, {acc[2]:.3f}]")
    print(f"Precision: {prec[0]:.3f} [{prec[1]:.3f}, {prec[2]:.3f}]")
    print(f"Recall   : {rec[0]:.3f} [{rec[1]:.3f}, {rec[2]:.3f}]")
    print(f"F1-score : {f1[0]:.3f} [{f1[1]:.3f}, {f1[2]:.3f}]")
    print(f"Kappa    : {kappa[0]:.3f} [{kappa[1]:.3f}, {kappa[2]:.3f}]")




Class 0
Accuracy : 0.970 [0.956, 0.983]
Precision: 0.937 [0.894, 0.975]
Recall   : 0.942 [0.903, 0.975]
F1-score : 0.939 [0.909, 0.966]
Kappa    : 0.919 [0.879, 0.954]

Class 1
Accuracy : 0.979 [0.967, 0.989]
Precision: 0.968 [0.938, 0.993]
Recall   : 0.949 [0.915, 0.980]
F1-score : 0.958 [0.935, 0.978]
Kappa    : 0.944 [0.913, 0.971]

Class 2
Accuracy : 0.940 [0.921, 0.957]
Precision: 0.874 [0.824, 0.921]
Recall   : 0.886 [0.833, 0.936]
F1-score : 0.880 [0.841, 0.915]
Kappa    : 0.839 [0.789, 0.886]

Class 3
Accuracy : 0.965 [0.949, 0.978]
Precision: 0.929 [0.886, 0.969]
Recall   : 0.930 [0.888, 0.969]
F1-score : 0.929 [0.897, 0.957]
Kappa    : 0.905 [0.864, 0.943]


In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score,
    recall_score, f1_score,
    cohen_kappa_score,
    confusion_matrix,
    classification_report
)
import numpy as np

def bootstrap_ci_multiclass(metric_fn, y_true, y_pred,
                            n_bootstrap=1000, seed=42):
    rng = np.random.default_rng(seed)
    scores = []

    for _ in range(n_bootstrap):
        idx = rng.choice(len(y_true), size=len(y_true), replace=True)
        scores.append(metric_fn(y_true[idx], y_pred[idx]))

    scores = np.array(scores)
    return (
        scores.mean(),
        np.percentile(scores, 2.5),
        np.percentile(scores, 97.5)
    )


In [ ]:
y_true = np.array(test.labels)
y_pred = np.array(pred_class_indices)

acc = bootstrap_ci_multiclass(accuracy_score, y_true, y_pred)

prec = bootstrap_ci_multiclass(
    lambda y, p: precision_score(y, p, average="macro", zero_division=0),
    y_true, y_pred
)

rec = bootstrap_ci_multiclass(
    lambda y, p: recall_score(y, p, average="macro", zero_division=0),
    y_true, y_pred
)

f1 = bootstrap_ci_multiclass(
    lambda y, p: f1_score(y, p, average="macro", zero_division=0),
    y_true, y_pred
)

kappa = bootstrap_ci_multiclass(
    cohen_kappa_score,
    y_true, y_pred
)


In [ ]:
print(f"Accuracy  : {acc[0]:.3f} [{acc[1]:.3f}, {acc[2]:.3f}]")
print(f"Precision : {prec[0]:.3f} [{prec[1]:.3f}, {prec[2]:.3f}]")
print(f"Recall    : {rec[0]:.3f} [{rec[1]:.3f}, {rec[2]:.3f}]")
print(f"F1-score  : {f1[0]:.3f} [{f1[1]:.3f}, {f1[2]:.3f}]")
print(f"Kappa     : {kappa[0]:.3f} [{kappa[1]:.3f}, {kappa[2]:.3f}]")

print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("\nClassification Report:\n",
      classification_report(y_true, y_pred, digits=3))


Accuracy  : 0.926 [0.907, 0.946]
Precision : 0.927 [0.907, 0.947]
Recall    : 0.926 [0.907, 0.947]
F1-score  : 0.926 [0.906, 0.947]
Kappa     : 0.902 [0.875, 0.928]

Confusion Matrix:
 [[149   0   6   3]
 [  0 150   6   2]
 [  9   3 140   6]
 [  1   2   8 147]]

Classification Report:
               precision    recall  f1-score   support

           0      0.937     0.943     0.940       158
           1      0.968     0.949     0.958       158
           2      0.875     0.886     0.881       158
           3      0.930     0.930     0.930       158

    accuracy                          0.927       632
   macro avg      0.928     0.927     0.927       632
weighted avg      0.928     0.927     0.927       632



#GRAD-CAM BATCH PROCESSING

In [ ]:
# =====================  =====================
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import gc


TEST_DIR = "F:/Willey/TGCA Extended Dataset-2/Test"
OUTPUT_DIR = "F:/Willey/Gradcam result"
IMG_SIZE = (224, 224)          # must match model input
ALPHA = 0.4
BATCH_SIZE = 1                 # adjust based on GPU memory
MAX_IMAGES_PER_CLASS = None    # None = all images

os.makedirs(OUTPUT_DIR, exist_ok=True)

target_conv_layer = None
for layer in fused_InceptionResNetV2.layers[::-1]:
    if isinstance(layer, tf.keras.layers.Conv2D):
        target_conv_layer = layer.name
        break
print("Using layer:", target_conv_layer)

# ---------------- GRAD-CAM MODEL ----------------
grad_model = tf.keras.models.Model(
    fused_InceptionResNetV2.input,
    [
        fused_InceptionResNetV2.get_layer(target_conv_layer).output,
        fused_InceptionResNetV2.output
    ]
)

# ---------------- FUNCTIONS ----------------
def load_image(path):
    """Load image and preprocess for InceptionResNetV2"""
    img = tf.keras.preprocessing.image.load_img(path, target_size=IMG_SIZE)
    img_arr = tf.keras.preprocessing.image.img_to_array(img)
    img_arr = np.expand_dims(img_arr, axis=0)
    img_arr = tf.keras.applications.inception_resnet_v2.preprocess_input(img_arr)
    return img_arr, np.array(img).astype("uint8")

def gradcam_batch(img_batch):
    """Compute Grad-CAM heatmaps for a batch"""
    with tf.GradientTape() as tape:
        conv_out, preds = grad_model(img_batch)
        class_idx = tf.argmax(preds, axis=-1)
        loss = tf.reduce_sum(tf.gather(preds, class_idx, axis=1, batch_dims=1))

    grads = tape.gradient(loss, conv_out)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    heatmaps = []

    for i in range(conv_out.shape[0]):
        heatmap = tf.reduce_sum(conv_out[i] * pooled_grads, axis=-1)
        heatmap = tf.maximum(heatmap, 0)
        heatmap /= tf.reduce_max(heatmap) + 1e-8
        heatmaps.append(heatmap.numpy())
    return heatmaps, class_idx.numpy()

def save_overlay(image, heatmap, path):
    """Overlay heatmap on image and save"""
    heatmap = tf.image.resize(
        heatmap[..., None], (image.shape[0], image.shape[1])
    ).numpy().squeeze()

    plt.figure(figsize=(4,4))
    plt.imshow(image)
    plt.imshow(heatmap, cmap='jet', alpha=ALPHA)
    plt.axis('off')
    plt.savefig(path, bbox_inches='tight', pad_inches=0)
    plt.close()

# ---------------- PROCESS ENTIRE TEST SET ----------------
for cls in os.listdir(TEST_DIR):
    cls_path = os.path.join(TEST_DIR, cls)
    if not os.path.isdir(cls_path):
        continue

    save_cls = os.path.join(OUTPUT_DIR, cls)
    os.makedirs(save_cls, exist_ok=True)
    print(f"\nProcessing class: {cls}")

    images = os.listdir(cls_path)
    if MAX_IMAGES_PER_CLASS:
        images = images[:MAX_IMAGES_PER_CLASS]

    # Batch processing
    for i in range(0, len(images), BATCH_SIZE):
        batch_images = images[i:i+BATCH_SIZE]
        img_tensors, orig_images = [], []

        for img_name in batch_images:
            img_path = os.path.join(cls_path, img_name)
            img_tensor, orig_img = load_image(img_path)
            img_tensors.append(img_tensor)
            orig_images.append(orig_img)

        img_tensors = np.vstack(img_tensors)  # shape: (batch_size, 224, 224, 3)
        heatmaps, preds = gradcam_batch(img_tensors)

        # Save overlays
        for j, img_name in enumerate(batch_images):
            save_path = os.path.join(
                save_cls,
                f"{os.path.splitext(img_name)[0]}_pred{preds[j]}.png"
            )
            save_overlay(orig_images[j], heatmaps[j], save_path)

        # --------- MEMORY CLEANUP ---------
        del img_tensors, orig_images, heatmaps
        tf.keras.backend.clear_session()
        gc.collect()

print("✅ Grad-CAM completed safely in batches")




Using layer: conv2d_363_mirror

Processing class: KICH
